# This code is being used to train a model using tweets/users attributes but not NLP or Image Processing

In [5]:
import MySQLdb
import pandas as pd
from Classes.Database import Database
import json
import urllib2
import numpy
import math
import time
import datetime
import os
import os.path
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import average_precision_score

In [6]:
def getimage(url, full_name):
    full_name = "images/" + full_name
    if os.path.isfile(full_name):
        return mpimg.imread(full_name)
    
    try:
        f = urllib2.urlopen(url)
    except:
        return None
    
    data = f.read()
    with open(full_name, "wb") as code:
        code.write(data)
    return mpimg.imread(full_name)
def getallimages(imagesList):
    downloaded_images = []
    for image in imagesList:
        img = getimage(image,image.split('/')[-1])
        downloaded_images.append(img)
    return downloaded_images

In [7]:
engine = Database()

In [8]:
users = pd.read_sql("SELECT screen_name, if(followers_count = 0, 1, followers_count) as followers_count, \
                if(friends_count = 0, 1, friends_count) as friends_count,\
                statuses_count, if(verified, 1, 0) as verified,\
                TIMESTAMPDIFF(YEAR,created_at,CURDATE()) as 'Account Age',\
                klout_score \
                FROM user WHERE klout_score is not null",
                    con = engine.myDB, index_col = 'screen_name')
users

,followers_count,friends_count,statuses_count,verified,Account Age,klout_score
screen_name,,,,,,
Lysette_P,420,401,4787,0,8,0.0
Gboarders,386,1021,5804,0,5,0.0
PabloAcostaTuc,1090,216,2371,0,5,0.0
patigalafassi,169,170,5386,0,8,0.0
tricky_dickie87,320,1344,663,0,5,0.0
joeflech,3658,1224,11410,1,8,0.0
MariaLaurita3p,244,648,2415,0,8,0.0
tjchev_51,469,518,3867,0,5,0.0
casadecrews,2980,1505,30852,0,8,0.0


In [9]:
users.describe()

,followers_count,friends_count,statuses_count,verified,Account Age,klout_score
count,9.613700e+04,96137.000000,9.613700e+04,96137.000000,96137.000000,96137.0
mean,3.758839e+03,764.862831,1.055268e+04,0.017600,5.590595,0.0
std,8.248914e+04,3458.788973,2.442905e+04,0.131493,2.512334,0.0
min,1.000000e+00,1.000000,1.000000e+00,0.000000,0.000000,0.0
25%,1.270000e+02,172.000000,7.730000e+02,0.000000,4.000000,0.0
50%,3.440000e+02,364.000000,3.171000e+03,0.000000,6.000000,0.0
75%,8.360000e+02,734.000000,1.069100e+04,0.000000,8.000000,0.0
max,1.178080e+07,455605.000000,2.021478e+06,1.000000,11.000000,0.0


In [10]:
print len(users.loc[users['followers_count'] < 5000])
print len(users.loc[users['friends_count'] < 100])
print len(users.loc[users['statuses_count'] < 100])

91189
13192
6414


In [15]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

myusers = users.copy()
numerical_cols = ['followers_count','friends_count','statuses_count','Account Age','klout_score']


scaler1 = StandardScaler()
scaler2 = StandardScaler()
myusers[numerical_cols] = scaler1.fit_transform(myusers[numerical_cols])
myusers[numerical_cols] = scaler2.fit_transform(myusers[numerical_cols])

In [16]:
myusers

,followers_count,friends_count,statuses_count,verified,Account Age,klout_score
screen_name,,,,,,
Lysette_P,-0.040476,-0.105200,-0.236019,0,0.959036,0.0
Gboarders,-0.040888,0.074054,-0.194388,0,-0.235079,0.0
PabloAcostaTuc,-0.032354,-0.158687,-0.334918,0,-0.235079,0.0
patigalafassi,-0.043519,-0.171987,-0.211498,0,0.959036,0.0
tricky_dickie87,-0.041689,0.167440,-0.404835,0,-0.235079,0.0
joeflech,-0.001222,0.132746,0.035094,1,0.959036,0.0
MariaLaurita3p,-0.042610,-0.033787,-0.333117,0,0.959036,0.0
tjchev_51,-0.039882,-0.071373,-0.273679,0,-0.235079,0.0
casadecrews,-0.009442,0.213988,0.830954,0,0.959036,0.0


In [93]:
# users['Our_Score'] = users['followers_count'] * 0.48 + users['friends_count'] * 0.26 + users['statuses_count'] * 0.26
skfolds = StratifiedKFold(n_splits=3, random_state=42)
train_set, test_set = train_test_split(myusers, test_size=0.2, random_state=42)
X_train = train_set.iloc[:,[0,1,2,4]]
Y_train = train_set['verified']

for train_index, test_index in skfolds.split(X_train, Y_train):
    forest_clf = RandomForestClassifier(random_state=42, n_estimators=30)
    X_train_folds = X_train.iloc[train_index]
    y_train_folds = (Y_train.iloc[train_index])
    X_test_fold = X_train.iloc[test_index]
    y_test_fold = (Y_train.iloc[test_index])
    
    forest_clf = forest_clf.fit(X_train,Y_train)
    y_pred = forest_clf.predict(X_test_fold)
    print(forest_clf.feature_importances_)
    print(confusion_matrix(y_test_fold, y_pred))
    print("Precision Score: ", precision_score(y_test_fold, y_pred))
    print("Recall Score:", recall_score(y_test_fold, y_pred))
    print("Average Precision Score", average_precision_score(y_test_fold, y_pred))
    

[0.46235234 0.24090841 0.23485754 0.06188171]
[[28494     0]
 [   11   512]]
('Precision Score: ', 1.0)
('Recall Score:', 0.9789674952198852)
('Average Precision Score', 0.9793465833406421)
[0.46235234 0.24090841 0.23485754 0.06188171]
[[28493     0]
 [   12   510]]
('Precision Score: ', 1.0)
('Recall Score:', 0.9770114942528736)
('Average Precision Score', 0.9774250734360547)
[0.46235234 0.24090841 0.23485754 0.06188171]
[[28492     1]
 [    9   513]]
('Precision Score: ', 0.9980544747081712)
('Recall Score:', 0.9827586206896551)
('Average Precision Score', 0.9811568233247264)


In [116]:
myusers['myScore'] = ((myusers['followers_count'] + abs(myusers.min()[0])) / \
                    (myusers['friends_count'] + abs(myusers.min()[1]) + 1)) * 55 + \
                    myusers['statuses_count'] * 35 + \
                    myusers['Account Age'] * 10
myusers

,followers_count,friends_count,statuses_count,verified,Account Age,klout_score,myScore
screen_name,,,,,,,
Bryce__Foster,-0.038808,-0.038400,-0.334552,0,-0.329500,0.128499,-14.647842
tmj_FLD_HRTA,-0.039532,-0.048665,-0.386069,0,0.997014,-0.735027,-3.221238
tmj_BHM_adm,-0.040450,-0.048010,-0.389247,0,0.997014,-1.659503,-3.382196
tmj_AZS_adm,-0.039991,-0.049174,-0.387764,0,0.997014,-0.552795,-3.305190
HIKARENLALA,-0.042675,-0.067520,-0.299399,0,-0.329500,-1.209943,-13.619669
tmj_TUL_adm,-0.040327,-0.048010,-0.389271,0,0.997014,-0.863020,-3.376356
tmj_KYL_retail,-0.039497,-0.046772,-0.384468,0,0.997014,-0.802185,-3.163894
tmj_MTP_retail,-0.039832,-0.047063,-0.389035,0,0.997014,-1.228296,-3.341743
todd_malone,-0.042340,-0.048810,-0.373872,0,-0.329500,-0.612102,-16.210950


In [117]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

train_set, test_set = train_test_split(myusers, test_size=0.2, random_state=42)

X_train = train_set.iloc[:,[0,1,2,6]]
Y_train = train_set['klout_score']
X_test = test_set.iloc[:,[0,1,2,6]]
Y_test = test_set['klout_score']

forest_clf = RandomForestRegressor(random_state=42, n_estimators=100,n_jobs=-1,min_samples_leaf=30)
forest_clf = forest_clf.fit(X_train,Y_train)
y_pred = forest_clf.predict(X_test)

print(forest_clf.feature_importances_)


[0.81863446 0.03604879 0.07890616 0.06641059]


In [118]:
r2_score(Y_test, y_pred)

0.559483929758272

In [ ]:
x = Y_test - y_pred
print("Accuracy : {0}%".format(len(x[numpy.absolute(x) == 0])*100.0 / len(x)))
print("Accuracy +- 1: {0}%".format(len(x[numpy.absolute(x) == 1])*100.0 / len(x)))
print("Accuracy +- 2: {0}%".format(len(x[numpy.absolute(x) == 2])*100.0 / len(x)))

print "    ",
for i in range(1,10):
    print "{0}   ".format(i),
print ""
print(confusion_matrix(Y_test, y_pred))

In [ ]:
import itertools
import numpy as np
import matplotlib.pyplot as plt

from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Compute confusion matrix
cnf_matrix = confusion_matrix(Y_test, y_pred)
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=range(1,10),
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=range(1,10), normalize=True,
                      title='Normalized confusion matrix')

plt.show()

In [92]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

train_set, test_set = train_test_split(users, test_size=0.2, random_state=42)

X_train = train_set.iloc[:,[0,1,2]]
Y_train = train_set['klout_score'] > 40
X_test = test_set.iloc[:,[0,1,2]]
Y_test = test_set['klout_score'] > 40

forest_clf = RandomForestClassifier(random_state=42, n_estimators=100,n_jobs=-1,min_samples_leaf=100,
                                    max_depth=3, min_impurity_decrease = 0.001)
forest_clf = forest_clf.fit(X_train,Y_train)
y_pred = forest_clf.predict(X_test)

print(forest_clf.feature_importances_)
print(confusion_matrix(Y_test, y_pred))
print("Precision Score: ", precision_score(Y_test, y_pred))
print("Recall Score:", recall_score(Y_test, y_pred))
print("Average Precision Score", average_precision_score(Y_test, y_pred))

[0.51 0.26 0.23]
[[7939 3365]
 [2710 7077]]
('Precision Score: ', 0.6777437272553151)
('Recall Score:', 0.7231020741800347)
('Average Precision Score', 0.6185687204114059)


In [180]:
tweetsDuringEvent = pd.read_sql("SELECT * FROM tweetCoords WHERE CREATED_AT >= DATE('2017-09-10') AND CREATED_AT <= DATE('2017-09-12')", myDB)

In [181]:
tweetsDuringEvent

,tweet_id,created_at,text,user_id,coordinates,location_coordinates,longitude,latitude,place_id,image_url,language,hashtags
0,906668545542680576,2017-09-10 00:00:01,"Wind 2.0 mph N. Barometer 29.880 in, Steady. T...",1227982520,"[28.87527778,-81.2525]","[[[-81.285896, 28.84511], [-81.285896, 28.9640...",-81.2525,28.8753,Deltona,None,en,None
1,906668555185291265,2017-09-10 00:00:03,There is always beauty regardless the circumst...,42239064,"[29.5381,-81.2234]","[[[-81.279617, 29.424649], [-81.279617, 29.627...",-81.2234,29.5381,Palm Coast,None,en,None
2,906668556493889536,2017-09-10 00:00:03,#Carpool #Orlando - gt #Orlando #KIrkmanRoad ...,87188071,"[28.4363045,-81.4797168]","[[[-87.634643, 24.396308], [-87.634643, 31.001...",-81.4797,28.4363,Florida,None,en,"Carpool,Orlando,Orlando,KIrkmanRoad"
3,906668570079309830,2017-09-10 00:00:06,@avictoria_x nah i m chillin my eyes never get...,746764728,None,"[[[-81.25235, 28.549308], [-81.25235, 28.58224...",-81.2380,28.5658,Union Park,None,en,None
4,906668576056246278,2017-09-10 00:00:08,@jvnvy @sza That s how I feel bout seein travis,1679199278,None,"[[[-82.31457, 27.823335], [-82.31457, 27.85374...",-82.2877,27.8385,Boyette,None,en,None
5,906668586793660416,2017-09-10 00:00:10,Taking a beak from boarding the windows. Kitty...,33160615,"[27.91,-82.35]","[[[-87.634643, 24.396308], [-87.634643, 31.001...",-82.3500,27.9100,Florida,None,en,None
6,906668589385764864,2017-09-10 00:00:11,"19:53 Temp. 82.4 F, Hum. 79 , Dewp. 73.6 F, Ba...",19511251,"[27.86977674,-82.80524377]","[[[-82.81611, 27.80979], [-82.81611, 27.884351...",-82.8052,27.8698,Seminole,None,en,None
7,906668590878871555,2017-09-10 00:00:11,"I m so proud of our local radio, news, animal ...",328572345,None,"[[[-82.771167, 27.839081], [-82.771167, 27.878...",-82.7544,27.8586,Bardmoor,None,en,IrmaHurricane2017
8,906668595358216192,2017-09-10 00:00:12,Evening of a Hurricane. #hurricane #hurricanei...,278782686,"[27.9709,-82.4646]","[[[-82.620093, 27.821353], [-82.620093, 28.171...",-82.4646,27.9709,Tampa,None,en,"hurricane,hurricaneirma,sunset,tampa,florida,w..."
9,906668601817669632,2017-09-10 00:00:14,hey big head https://t.co/M46uNJoxx2,2661138289,None,"[[[-80.352067, 25.923399], [-80.352067, 25.957...",-80.3226,25.9403,Country Club,None,en,None


In [182]:
allImages = getallimages(tweetsDuringEvent.loc[tweetsDuringEvent['image_url'].notnull()]['image_url'].values)

In [20]:
newUsers = pd.read_sql("SELECT user_id, followers_count, friends_count, statuses_count, idsc.score, verified, infes.*, infrs.* FROM myInfluencees infes JOIN myInfluencers infrs on infes.user = infrs.user JOIN klout_id_score idsc on infes.user = idsc.id JOIN user u on u.screen_name = idsc.screen_name where idsc.screen_name in (SELECT screen_name from user);", myDB, index_col='user_id')

In [21]:
newUsers = newUsers.drop(axis=1, columns='user')

In [22]:
newUsers

,followers_count,friends_count,statuses_count,score,verified,influencees_std,influencees_avg,influencees_max,influencees_min,influencees_count,influencers_std,influencers_avg,influencers_max,influencers_min,influencers_count
user_id,,,,,,,,,,,,,,,
1000029698,379,411,2332,38.379501,0,6.21,33.33,41.86,22.76,5,10.41,57.07,70.67,41.86,5
100003413,286,279,9,19.431200,0,8.36,49.40,58.49,34.62,5,16.45,75.32,98.90,58.49,5
100003539,312,263,72,31.159500,0,10.05,28.20,43.13,17.55,5,4.55,71.82,78.54,64.21,5
1000061430,160,11,3825,24.195400,0,17.95,49.77,83.59,33.08,5,6.54,88.73,97.88,80.51,5
100008086,293,279,8,27.871901,0,9.09,34.30,48.33,21.57,5,3.77,65.70,70.94,60.02,5
1000089342,179,268,662,30.531000,0,6.69,49.55,60.48,40.90,5,6.54,53.55,61.69,45.91,5
1000093178,319,470,183,26.388700,0,9.03,45.09,58.34,35.56,5,16.44,73.84,92.08,52.69,5
1000120868,366,285,23979,41.343899,0,9.17,37.69,50.65,24.56,5,22.13,66.18,84.40,24.56,5
1000132536,22805,18986,1393,41.569901,0,5.49,46.84,53.29,39.24,5,17.19,62.38,83.26,45.03,5


In [12]:
newUsers.describe()

,followers_count,friends_count,statuses_count,verified,influencees_count,influencers_count
count,8.507800e+04,8.507800e+04,8.507800e+04,85078.000000,85078.000000,85078.000000
mean,2.890145e+03,1.024349e+03,1.788785e+04,0.020170,4.682938,4.942817
std,6.110665e+04,1.589826e+04,4.512953e+04,0.140581,2.271799,0.916285
min,0.000000e+00,0.000000e+00,1.000000e+00,0.000000,1.000000,1.000000
25%,2.410000e+02,2.640000e+02,2.507000e+03,0.000000,5.000000,5.000000
50%,5.190000e+02,4.900000e+02,7.801500e+03,0.000000,5.000000,5.000000
75%,1.105000e+03,9.190000e+02,2.025100e+04,0.000000,5.000000,5.000000
max,1.312178e+07,4.358008e+06,9.395746e+06,1.000000,342.000000,61.000000


In [23]:
newUsers.corr()

,followers_count,friends_count,statuses_count,score,verified,influencees_count,influencers_count
followers_count,1.000000,0.278782,0.025744,0.107872,0.164156,0.117712,-0.001530
friends_count,0.278782,1.000000,0.039508,0.055593,0.048354,0.011719,0.000789
statuses_count,0.025744,0.039508,1.000000,0.219076,0.037105,0.053792,0.053411
score,0.107872,0.055593,0.219076,1.000000,0.297336,0.235171,0.051058
verified,0.164156,0.048354,0.037105,0.297336,1.000000,0.098636,-0.005190
influencees_count,0.117712,0.011719,0.053792,0.235171,0.098636,1.000000,0.243507
influencers_count,-0.001530,0.000789,0.053411,0.051058,-0.005190,0.243507,1.000000


In [24]:
xUsers = newUsers.copy()

In [25]:
xUsers[xUsers.columns[5:]] = xUsers[xUsers.columns[5:]].astype(float)

In [26]:
xUsers.corr()

,followers_count,friends_count,statuses_count,score,verified,influencees_std,influencees_avg,influencees_max,influencees_min,influencees_count,influencers_std,influencers_avg,influencers_max,influencers_min,influencers_count
followers_count,1.000000,0.278782,0.025744,0.107872,0.164156,0.012754,0.002621,0.017096,-0.011650,0.117712,-0.008412,0.002655,-0.001650,0.007776,-0.001530
friends_count,0.278782,1.000000,0.039508,0.055593,0.048354,0.005527,0.017770,0.019143,0.008173,0.011719,-0.006061,0.001549,-0.002180,0.003806,0.000789
statuses_count,0.025744,0.039508,1.000000,0.219076,0.037105,-0.047870,0.100407,0.056050,0.102066,0.053792,-0.038967,-0.029821,-0.039301,0.001227,0.053411
score,0.107872,0.055593,0.219076,1.000000,0.297336,0.012966,0.331899,0.274787,0.212845,0.235171,-0.142359,-0.089121,-0.114463,0.041048,0.051058
verified,0.164156,0.048354,0.037105,0.297336,1.000000,0.049422,0.062244,0.090512,0.011149,0.098636,-0.030217,0.008435,-0.000479,0.032824,-0.005190
influencees_std,0.012754,0.005527,-0.047870,0.012966,0.049422,1.000000,0.014825,0.580471,-0.567946,0.163751,0.101332,0.168769,0.191061,0.060361,0.175886
influencees_avg,0.002621,0.017770,0.100407,0.331899,0.062244,0.014825,1.000000,0.773353,0.759275,0.085227,-0.140265,0.072372,0.012738,0.163550,0.061528
influencees_max,0.017096,0.019143,0.056050,0.274787,0.090512,0.580471,0.773353,1.000000,0.310814,0.192056,-0.045379,0.153860,0.122658,0.157788,0.155065
influencees_min,-0.011650,0.008173,0.102066,0.212845,0.011149,-0.567946,0.759275,0.310814,1.000000,-0.071473,-0.162397,-0.037685,-0.098395,0.089648,-0.072817
influencees_count,0.117712,0.011719,0.053792,0.235171,0.098636,0.163751,0.085227,0.192056,-0.071473,1.000000,0.043415,0.037642,0.071861,0.010043,0.243507


In [118]:
tweetText = pd.read_sql("SELECT tweet_id,text from tweetCoords", getDBInstance('geotwitter'), index_col='tweet_id')

In [188]:
tweetText.to_csv("tweetsText.csv")

In [122]:
myTweets = pd.read_sql("SELECT * FROM polished_tweets", getDBInstance('geotwitter'), index_col='tweet_id')

In [176]:
import re
def getLinks(texts):
    ans = []
    for text in texts:
        ans.append(len(re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', text)) != 0)
    return ans

In [177]:
myTweets['has_url'] = getLinks(myTweets['text'])

In [180]:
myTweets.has_hashtags = myTweets.has_hashtags == 1

In [181]:
myTweets.has_image = myTweets.has_image == 1

In [183]:
myTweets = myTweets[['text','created_at','user_id','latitude','longitude','has_image','has_hashtags','has_url','image_url']]

In [186]:
myTweets = myTweets.drop(axis=1, columns=['text','created_at','user_id','latitude','longitude','image_url'])

In [194]:
myTweets.to_sql(con=engine,name='tweetFeatures',index=False)

In [191]:
myTweets['tweet_id'] = myTweets.index